In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from PIL import Image
from utils.camera import Camera
import warnings
warnings.filterwarnings('ignore')

In [2]:
#base_model_class = keras.applications.MobileNetV2
#base_model_preprocess_function = keras.applications.mobilenet_v2.preprocess_input

base_model_class = keras.applications.InceptionV3
base_model_preprocess_function = keras.applications.inception_v3.preprocess_input

In [3]:
model = keras.models.load_model(f'../models/model_fingers_{base_model_class.__name__}')

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,288,384] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Add]

## predict from camera

In [ ]:
# initialize only one time
if 'camera0' not in locals():
    camera0 = Camera(capture_device=0, width=224, height=224, 
                     capture_width=1280, capture_height=720, capture_fps=30, flip_mode=-1, debug=True)

In [ ]:
img = camera0.get_image_rgb()
img = Image.fromarray(img, 'RGB')
img_tensor, duration_preprocess = preprocess_image(img)
df, duration_predict = predict_from_image_tensor(img_tensor)
pred_label = df.nlargest(1, 'score')[['label','score']].values[0][0].split(',')[0]
print(f'{pred_label} - time preprocess / prediction:{duration_preprocess:0.2f}s / {duration_predict:0.2f}s')

## live camera

In [ ]:
duration = 10 # running time in seconds

from IPython.display import display
import ipywidgets.widgets as widgets
image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.Text(description='detected')
confidence_widget = widgets.Text(description='score')
time_widget = widgets.Text(description='pred. time')
progress_widget = widgets.FloatProgress(value=0, min=0.0, max=float(duration), description='running time')

display(widgets.HBox([image_widget, 
                      widgets.VBox([label_widget, confidence_widget, time_widget, progress_widget])]))

start_time = time.time()
while time.time() - start_time < duration:
    img_rgb = camera0.get_image_rgb()
    img = Image.fromarray(img_rgb, 'RGB')
    img_tensor, duration_preprocess = preprocess_image(img)
    df, duration_predict = predict_from_image_tensor(img_tensor)
    pred_label = df.nlargest(1, 'score')[['label','score']].values[0][0].split(',')[0]
    pred_proba = df.nlargest(1, 'score')[['label','score']].values[0][1]
    label_widget.value = pred_label
    confidence_widget.value = f'{pred_proba:0.0f}'
    time_widget.value = f'{duration_predict:0.2f} sec'
    image_widget.value = bytes(cv2.imencode('.jpg', img_rgb)[1])
    progress_widget.value = (time.time() - start_time)